RECIPES AND RATINGS

**Name(s)**: Nabila Afifah Qotrunnada

**Website Link**: nabilafifahq.github.io/recipes-ratings

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import random
import ast
import plotly.io as pio
import plotly.graph_objects as go
import os
import plotly.express as px
pd.options.plotting.backend = 'plotly'
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

custom_template = go.layout.Template(
    layout=go.Layout(
        font=dict(
            family="Monospace",
            size=14,
            color="#1B1F23"
        ),
        colorway=["#FF7F3E"],
        plot_bgcolor="white",
        paper_bgcolor="white",

        title=dict(
            font=dict(
                family="Monospace",
                size=20,
                color="#4477CE"
            ),
            x=0.5,
            xanchor="center"
        ),

        xaxis=dict(
            showgrid=True,
            gridcolor="#E0E0E0",
            gridwidth=1,
            zeroline=False,
            ticks="outside",
            tickcolor="#FF7F3E",
            ticklen=5,
            title_font=dict(
                family="Monospace",
                size=16,
                color="#FF7F3E"
            ),
            tickfont=dict(
                family="Monospace",
                size=12,
                color="#FF7F3E"
            ),
            linecolor="#B0B0B0",
            linewidth=1
        ),

        yaxis=dict(
            showgrid=True,
            gridcolor="#E0E0E0",
            gridwidth=1,
            zeroline=False,
            ticks="outside",
            tickcolor="#4477CE",
            ticklen=5,
            title_font=dict(
                family="Monospace",
                size=16,
                color="#4477CE"
            ),
            tickfont=dict(
                family="Monospace",
                size=12,
                color="#4477CE"
            ),
            linecolor="#B0B0B0",
            linewidth=1
        ),

        legend=dict(
            font=dict(
                family="Monospace",
                size=12,
                color="#1B1F23"
            ),
            bordercolor="#E0E0E0",
            borderwidth=1,
            bgcolor="white"
        )
    )
)

pio.templates["user_friendly_axes"] = custom_template
pio.templates.default = "user_friendly_axes"

from IPython.display import Markdown, display

os.makedirs("assets/plots", exist_ok=True)
os.makedirs("assets/tables", exist_ok=True)

def save_plotly(fig, name):
    filepath = f"assets/plots/{name}"
    fig.write_html(filepath, include_plotlyjs="cdn")
    print(f"Saved {filepath}")

## Step 1: Introduction

In [2]:
# It’s Food.com recipes (RAW_recipes.csv) plus user ratings (interactions.csv) since 2008.
# I merged them to get an avg_rating per recipe.
#
# Central Question:
#   Which recipe characteristics (prep time, nutrition profile, tags) are associated with higher average ratings?
#
# We will focus first on:
#   1. How does prep time (`minutes`) relate to `avg_rating`?
#   2. How do nutrition features (e.g. calories, protein) relate to `avg_rating`?
#   3. How do tags (e.g., “desserts,” “quick meals”) relate to `avg_rating`?
#
# First, load and merge the raw data.

# Step 1.1: Load RAW_recipes.csv and interactions.csv
recipes_raw = pd.read_csv("data/RAW_recipes.csv")
interactions_raw = pd.read_csv("data/interactions.csv")

# Display first few rows to verify
print("=== RAW_recipes.csv (first 5 rows) ===")
display(recipes_raw.head())
print("\n=== interactions.csv (first 5 rows) ===")
display(interactions_raw.head())

# Print shapes
print(f"\nRAW_recipes.csv shape: {recipes_raw.shape}")
print(f"interactions.csv shape: {interactions_raw.shape}")

# Step 1.2: Merge on recipe ID (left merge: keep all recipes)
merged = recipes_raw.merge(
    interactions_raw,
    left_on="id",
    right_on="recipe_id",
    how="left",
    suffixes=("_recipe", "_inter")
)

# Step 1.3: Replace rating == 0 with NaN (0 means "no rating")
merged["rating"] = merged["rating"].replace(0, np.nan)

# Step 1.4: Compute average rating per recipe
avg_ratings = merged.groupby("id")["rating"].mean().rename("avg_rating")

# Step 1.5: Merge avg_rating back into recipes_raw
recipes = recipes_raw.merge(
    avg_ratings,
    left_on="id",
    right_index=True,
    how="left"
)

# Display first 5 rows of merged DataFrame with avg_rating
print("\n=== recipes (with avg_rating) – first 5 rows ===")
display(recipes.head())

# Report the number of rows and relevant columns
print(f"\nrecipes DataFrame shape: {recipes.shape}")
print("Relevant columns for our analysis:")
print(" - minutes: prep time in minutes")
print(" - nutrition: string-encoded list of [calories, total_fat, sugar, sodium, protein, saturated_fat, carbohydrates]")
print(" - tags: string-encoded list of recipe tags (e.g., 'desserts', 'quick meals')")
print(" - ingredients, n_ingredients: list of ingredients and count")
print(" - avg_rating: average user rating (1–5)")

=== RAW_recipes.csv (first 5 rows) ===


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,1 brownies in the world best ever,333281,40,985201,2008-10-27,"['60-minutes-or-less', 'time-to-make', 'course...","[138.4, 10.0, 50.0, 3.0, 3.0, 19.0, 6.0]",10,['heat the oven to 350f and arrange the rack i...,"these are the most; chocolatey, moist, rich, d...","['bittersweet chocolate', 'unsalted butter', '...",9
1,1 in canada chocolate chip cookies,453467,45,1848091,2011-04-11,"['60-minutes-or-less', 'time-to-make', 'cuisin...","[595.1, 46.0, 211.0, 22.0, 13.0, 51.0, 26.0]",12,"['pre-heat oven the 350 degrees f', 'in a mixi...",this is the recipe that we use at my school ca...,"['white sugar', 'brown sugar', 'salt', 'margar...",11
2,412 broccoli casserole,306168,40,50969,2008-05-30,"['60-minutes-or-less', 'time-to-make', 'course...","[194.8, 20.0, 6.0, 32.0, 22.0, 36.0, 3.0]",6,"['preheat oven to 350 degrees', 'spray a 2 qua...",since there are already 411 recipes for brocco...,"['frozen broccoli cuts', 'cream of chicken sou...",9
3,millionaire pound cake,286009,120,461724,2008-02-12,"['time-to-make', 'course', 'cuisine', 'prepara...","[878.3, 63.0, 326.0, 13.0, 20.0, 123.0, 39.0]",7,"['freheat the oven to 300 degrees', 'grease a ...",why a millionaire pound cake? because it's su...,"['butter', 'sugar', 'eggs', 'all-purpose flour...",7
4,2000 meatloaf,475785,90,2202916,2012-03-06,"['time-to-make', 'course', 'main-ingredient', ...","[267.0, 30.0, 12.0, 12.0, 29.0, 48.0, 2.0]",17,"['pan fry bacon , and set aside on a paper tow...","ready, set, cook! special edition contest entr...","['meatloaf mixture', 'unsmoked bacon', 'goat c...",13



=== interactions.csv (first 5 rows) ===


,user_id,recipe_id,date,rating,review
0,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
1,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
2,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
3,124416,120345,2011-08-06,0,"Just an observation, so I will not rate. I fo..."
4,2000192946,120345,2015-05-10,2,This recipe was OVERLY too sweet. I would sta...



RAW_recipes.csv shape: (83782, 12)
interactions.csv shape: (731927, 5)

=== recipes (with avg_rating) – first 5 rows ===


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,avg_rating
0,1 brownies in the world best ever,333281,40,985201,2008-10-27,"['60-minutes-or-less', 'time-to-make', 'course...","[138.4, 10.0, 50.0, 3.0, 3.0, 19.0, 6.0]",10,['heat the oven to 350f and arrange the rack i...,"these are the most; chocolatey, moist, rich, d...","['bittersweet chocolate', 'unsalted butter', '...",9,4.0
1,1 in canada chocolate chip cookies,453467,45,1848091,2011-04-11,"['60-minutes-or-less', 'time-to-make', 'cuisin...","[595.1, 46.0, 211.0, 22.0, 13.0, 51.0, 26.0]",12,"['pre-heat oven the 350 degrees f', 'in a mixi...",this is the recipe that we use at my school ca...,"['white sugar', 'brown sugar', 'salt', 'margar...",11,5.0
2,412 broccoli casserole,306168,40,50969,2008-05-30,"['60-minutes-or-less', 'time-to-make', 'course...","[194.8, 20.0, 6.0, 32.0, 22.0, 36.0, 3.0]",6,"['preheat oven to 350 degrees', 'spray a 2 qua...",since there are already 411 recipes for brocco...,"['frozen broccoli cuts', 'cream of chicken sou...",9,5.0
3,millionaire pound cake,286009,120,461724,2008-02-12,"['time-to-make', 'course', 'cuisine', 'prepara...","[878.3, 63.0, 326.0, 13.0, 20.0, 123.0, 39.0]",7,"['freheat the oven to 300 degrees', 'grease a ...",why a millionaire pound cake? because it's su...,"['butter', 'sugar', 'eggs', 'all-purpose flour...",7,5.0
4,2000 meatloaf,475785,90,2202916,2012-03-06,"['time-to-make', 'course', 'main-ingredient', ...","[267.0, 30.0, 12.0, 12.0, 29.0, 48.0, 2.0]",17,"['pan fry bacon , and set aside on a paper tow...","ready, set, cook! special edition contest entr...","['meatloaf mixture', 'unsmoked bacon', 'goat c...",13,5.0



recipes DataFrame shape: (83782, 13)
Relevant columns for our analysis:
 - minutes: prep time in minutes
 - nutrition: string-encoded list of [calories, total_fat, sugar, sodium, protein, saturated_fat, carbohydrates]
 - tags: string-encoded list of recipe tags (e.g., 'desserts', 'quick meals')
 - ingredients, n_ingredients: list of ingredients and count
 - avg_rating: average user rating (1–5)


## Step 2: Data Cleaning and Exploratory Data Analysis

In [3]:
# 2.1 Parse 'nutrition' into numeric columns via ast.literal_eval
def parse_nutrition(nut_str):
    try:
        return ast.literal_eval(nut_str)
    except Exception:
        return [np.nan] * 7

nut_cols = [
    "calories", "total_fat", "sugar",
    "sodium", "protein", "saturated_fat", "carbohydrates"
]
recipes[nut_cols] = recipes["nutrition"].apply(parse_nutrition).tolist()

# 2.2 Parse 'tags' into Python lists via ast.literal_eval
def parse_tags(tags_str):
    try:
        return ast.literal_eval(tags_str)
    except Exception:
        return []

recipes["tags"] = recipes["tags"].apply(parse_tags)

# Verify that tags parsed correctly for a known recipe ID
print("=== Sample tags for recipe ID 333281 ===")
print(recipes.loc[recipes["id"] == 333281, "tags"].iloc[0])

# 2.3 Filter out unrealistic 'minutes' outliers (e.g., >1440 minutes)
recipes = recipes[recipes["minutes"] <= 1440].copy()

# 2.4 Summary of 'minutes'
print("\nSummary of 'minutes' after dropping >1440:")
print(recipes["minutes"].describe())

# 2.5 Check missing values in nutrition columns
print("\nMissing values by nutrition column:")
for col in nut_cols:
    print(f"  {col}: {recipes[col].isna().sum()}")

# 2.6 Count how many recipes have empty tags
empty_tags_count = recipes["tags"].apply(lambda L: len(L) == 0).sum()
print(f"\nNumber of recipes with empty 'tags': {empty_tags_count}")

# Display head of cleaned DataFrame (for website)
print("\n=== Head of cleaned DataFrame ===")
display(recipes[[
    "name", "id", "minutes", "contributor_id", "submitted",
    "tags", "nutrition", "n_steps", "steps", "description",
    "ingredients", "n_ingredients", "avg_rating",
    "calories", "total_fat", "sugar", "sodium",
    "protein", "saturated_fat", "carbohydrates"
]].head())
# 2.7 Univariate Analysis: two distribution plots

# (a) Histogram of prep time
fig_prep = px.histogram(
    recipes,
    x="minutes",
    nbins=50,
    title="Distribution of Recipe Prep Time",
    labels={"minutes": "Prep Time (minutes)", "count": "Number of Recipes"},
)
fig_prep.update_layout(
    title_font=dict(size=24),
    xaxis=dict(title_font=dict(size=18), tickfont=dict(size=14)),
    yaxis=dict(title_font=dict(size=18), tickfont=dict(size=14))
)
fig_prep.write_html("assets/plots/step2_prep_time.html", include_plotlyjs="cdn")
print("Saved assets/plots/step2_prep_time.html")

# (b) Histogram of average rating (drop NaN)
fig_rating = px.histogram(
    recipes.dropna(subset=["avg_rating"]),
    x="avg_rating",
    nbins=20,
    title="Distribution of Recipe Average Ratings",
    labels={"avg_rating": "Average Rating (stars)", "count": "Number of Recipes"},
)
fig_rating.update_layout(
    title_font=dict(size=24),
    xaxis=dict(title_font=dict(size=18), tickfont=dict(size=14)),
    yaxis=dict(title_font=dict(size=18), tickfont=dict(size=14))
)
fig_rating.write_html("assets/plots/step2_avg_rating.html", include_plotlyjs="cdn")
print("Saved assets/plots/step2_avg_rating.html")
# 2.8 Bivariate Analysis: two relationship plots

bivar_df = recipes.dropna(subset=["avg_rating"]).copy()

# (a) Scatter plot with LOWESS trendline: minutes vs. avg_rating
fig_scatter = px.scatter(
    bivar_df,
    x="minutes",
    y="avg_rating",
    trendline="lowess",
    title="Prep Time vs. Average Rating",
    labels={"minutes": "Prep Time (minutes)", "avg_rating": "Average Rating (stars)"},
)
fig_scatter.update_layout(
    title_font=dict(size=24),
    xaxis=dict(title_font=dict(size=18), tickfont=dict(size=14)),
    yaxis=dict(title_font=dict(size=18), tickfont=dict(size=14))
)
fig_scatter.write_html("assets/plots/step2_scatter_minutes_vs_rating.html", include_plotlyjs="cdn")
print("Saved assets/plots/step2_scatter_minutes_vs_rating.html")

# (b) Box plot: avg_rating by calorie bin
bivar_df["cal_bin"] = pd.cut(
    bivar_df["calories"],
    bins=[0, 200, 400, 600, 800, 1000, np.inf],
    labels=["0–200", "200–400", "400–600", "600–800", "800–1000", "1000+"],
    include_lowest=True
)
fig_box_cal = px.box(
    bivar_df,
    x="cal_bin",
    y="avg_rating",
    title="Average Rating by Calorie Range",
    labels={"cal_bin": "Calories (binned)", "avg_rating": "Average Rating (stars)"},
)
fig_box_cal.update_layout(
    title_font=dict(size=24),
    xaxis=dict(title_font=dict(size=18), tickfont=dict(size=14)),
    yaxis=dict(title_font=dict(size=18), tickfont=dict(size=14)),
    showlegend=False
)
fig_box_cal.write_html("assets/plots/step2_box_calories_vs_rating.html", include_plotlyjs="cdn")
print("Saved assets/plots/step2_box_calories_vs_rating.html")
# 2.9 Interesting Aggregates: grouped tables printed (for website screenshots)

# (a) Top 10 contributors by mean avg_rating (min 5 recipes)
contrib_summary = (
    bivar_df.groupby("contributor_id")
           .agg(mean_rating=("avg_rating", "mean"),
                n_recipes=("id", "count"))
           .reset_index()
           .sort_values(by="mean_rating", ascending=False)
)
top_contribs = contrib_summary[contrib_summary["n_recipes"] >= 5].head(10)
print("Top 10 Contributors by Mean Recipe Rating (min 5 recipes):")
display(top_contribs)

# (b) Average Rating by Number of Ingredients (binned)
bivar_df["n_ing_bin"] = pd.cut(
    bivar_df["n_ingredients"],
    bins=[0, 5, 10, 15, 20, np.inf],
    labels=["0–5", "5–10", "10–15", "15–20", "20+"],
    include_lowest=True
)
ingred_summary = (
    bivar_df.groupby("n_ing_bin", observed=True)["avg_rating"]
           .mean()
           .reset_index()
           .rename(columns={"avg_rating": "mean_avg_rating"})
)
print("\nAverage Rating by Number of Ingredients (binned):")
display(ingred_summary)

=== Sample tags for recipe ID 333281 ===
['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'for-large-groups', 'desserts', 'lunch', 'snacks', 'cookies-and-brownies', 'chocolate', 'bar-cookies', 'brownies', 'number-of-servings']

Summary of 'minutes' after dropping >1440:
count    83194.000000
mean        62.214270
std         97.362713
min          0.000000
25%         20.000000
50%         35.000000
75%         60.000000
max       1440.000000
Name: minutes, dtype: float64

Missing values by nutrition column:
  calories: 0
  total_fat: 0
  sugar: 0
  sodium: 0
  protein: 0
  saturated_fat: 0
  carbohydrates: 0

Number of recipes with empty 'tags': 0

=== Head of cleaned DataFrame ===


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,avg_rating,calories,total_fat,sugar,sodium,protein,saturated_fat,carbohydrates
0,1 brownies in the world best ever,333281,40,985201,2008-10-27,"[60-minutes-or-less, time-to-make, course, mai...","[138.4, 10.0, 50.0, 3.0, 3.0, 19.0, 6.0]",10,['heat the oven to 350f and arrange the rack i...,"these are the most; chocolatey, moist, rich, d...","['bittersweet chocolate', 'unsalted butter', '...",9,4.0,138.4,10.0,50.0,3.0,3.0,19.0,6.0
1,1 in canada chocolate chip cookies,453467,45,1848091,2011-04-11,"[60-minutes-or-less, time-to-make, cuisine, pr...","[595.1, 46.0, 211.0, 22.0, 13.0, 51.0, 26.0]",12,"['pre-heat oven the 350 degrees f', 'in a mixi...",this is the recipe that we use at my school ca...,"['white sugar', 'brown sugar', 'salt', 'margar...",11,5.0,595.1,46.0,211.0,22.0,13.0,51.0,26.0
2,412 broccoli casserole,306168,40,50969,2008-05-30,"[60-minutes-or-less, time-to-make, course, mai...","[194.8, 20.0, 6.0, 32.0, 22.0, 36.0, 3.0]",6,"['preheat oven to 350 degrees', 'spray a 2 qua...",since there are already 411 recipes for brocco...,"['frozen broccoli cuts', 'cream of chicken sou...",9,5.0,194.8,20.0,6.0,32.0,22.0,36.0,3.0
3,millionaire pound cake,286009,120,461724,2008-02-12,"[time-to-make, course, cuisine, preparation, o...","[878.3, 63.0, 326.0, 13.0, 20.0, 123.0, 39.0]",7,"['freheat the oven to 300 degrees', 'grease a ...",why a millionaire pound cake? because it's su...,"['butter', 'sugar', 'eggs', 'all-purpose flour...",7,5.0,878.3,63.0,326.0,13.0,20.0,123.0,39.0
4,2000 meatloaf,475785,90,2202916,2012-03-06,"[time-to-make, course, main-ingredient, prepar...","[267.0, 30.0, 12.0, 12.0, 29.0, 48.0, 2.0]",17,"['pan fry bacon , and set aside on a paper tow...","ready, set, cook! special edition contest entr...","['meatloaf mixture', 'unsmoked bacon', 'goat c...",13,5.0,267.0,30.0,12.0,12.0,29.0,48.0,2.0


Saved assets/plots/step2_prep_time.html
Saved assets/plots/step2_avg_rating.html
Saved assets/plots/step2_scatter_minutes_vs_rating.html
Saved assets/plots/step2_box_calories_vs_rating.html
Top 10 Contributors by Mean Recipe Rating (min 5 recipes):


,contributor_id,mean_rating,n_recipes
4466,467583,5.0,5
4404,461099,5.0,5
4321,452749,5.0,5
4365,456641,5.0,5
13822,2175724,5.0,5
4862,512696,5.0,5
4882,514811,5.0,5
8094,847054,5.0,7
8387,876513,5.0,6
4775,502938,5.0,5



Average Rating by Number of Ingredients (binned):


,n_ing_bin,mean_avg_rating
0,0–5,4.647109
1,5–10,4.616601
2,10–15,4.622795
3,15–20,4.637352
4,20+,4.701360


## Step 3: Assessment of Missingness

In [4]:
# 3.1 NMAR Analysis (narrative, no code needed)
# “avg_rating” is missing when a recipe never received any rating. 
# Because missingness depends on whether users viewed/rated a recipe (unobserved), 
# missingness is Not Missing At Random (NMAR).

# 3.2 Flag recipes where avg_rating is missing
recipes["is_missing_avg"] = recipes["avg_rating"].isna()

# 3.3 Permutation test: Does missingness of avg_rating depend on prep time?
obs_diff_minutes = (
    recipes.loc[recipes["is_missing_avg"], "minutes"].mean()
    - recipes.loc[~recipes["is_missing_avg"], "minutes"].mean()
)

n_perms = 5000
perm_diffs_minutes = np.zeros(n_perms)
minutes_vals = recipes["minutes"].values
flags_missing = recipes["is_missing_avg"].values
rng = np.random.default_rng(0)

for i in range(n_perms):
    shuffled = rng.permutation(flags_missing)
    m_missing = minutes_vals[shuffled].mean()
    m_not = minutes_vals[~shuffled].mean()
    perm_diffs_minutes[i] = m_missing - m_not

p_val_minutes = np.mean(np.abs(perm_diffs_minutes) >= np.abs(obs_diff_minutes))
print(f"Observed difference in minutes (missing – not) = {obs_diff_minutes:.2f}")
print(f"P-value (minutes) = {p_val_minutes:.4f}")

# 3.4 Permutation test: Does missingness of avg_rating depend on calories?
obs_diff_calories = (
    recipes.loc[recipes["is_missing_avg"], "calories"].mean()
    - recipes.loc[~recipes["is_missing_avg"], "calories"].mean()
)

perm_diffs_calories = np.zeros(n_perms)
cal_vals = recipes["calories"].values

for i in range(n_perms):
    shuffled = rng.permutation(flags_missing)
    m_missing_c = cal_vals[shuffled].mean()
    m_not_c = cal_vals[~shuffled].mean()
    perm_diffs_calories[i] = m_missing_c - m_not_c

p_val_calories = np.mean(np.abs(perm_diffs_calories) >= np.abs(obs_diff_calories))
print(f"\nObserved difference in calories (missing – not) = {obs_diff_calories:.2f}")
print(f"P-value (calories) = {p_val_calories:.4f}")

# 3.5 Visualize null distribution for 'minutes' permutation test
fig_missing_min = px.histogram(
    x=perm_diffs_minutes,
    nbins=50,
    title="Null Distribution: Difference in Mean Minutes (missing – not)",
    labels={"x": "Permuted difference", "y": "Count"},
)
fig_missing_min.add_vline(
    x=obs_diff_minutes,
    line_color="red",
    line_dash="dash",
    annotation_text="Observed diff",
    annotation_position="top right"
)
fig_missing_min.update_layout(
    title_font=dict(size=24),
    xaxis=dict(title_font=dict(size=18), tickfont=dict(size=14)),
    yaxis=dict(title_font=dict(size=18), tickfont=dict(size=14))
)
fig_missing_min.write_html("assets/plots/step3_null_minutes.html", include_plotlyjs="cdn")
print("Saved assets/plots/step3_null_minutes.html")

Observed difference in minutes (missing – not) = 13.75
P-value (minutes) = 0.0000

Observed difference in calories (missing – not) = 84.85
P-value (calories) = 0.0000
Saved assets/plots/step3_null_minutes.html


## Step 4: Hypothesis Testing

In [5]:
# Question: Do “quick” recipes (prep time ≤ 30 minutes) and “long” recipes (prep time > 30 minutes)
# have different average ratings?

# Null Hypothesis (H0): μ_quick = μ_long
# Alternative Hypothesis (H1): μ_quick ≠ μ_long

# 4.1 Define quick/long subset where avg_rating is not missing
rated = recipes.dropna(subset=["avg_rating"]).copy()
rated["quick"] = rated["minutes"] <= 30

mean_quick = rated.loc[rated["quick"], "avg_rating"].mean()
mean_long  = rated.loc[~rated["quick"], "avg_rating"].mean()
obs_diff   = mean_quick - mean_long

# 4.2 Permutation test for avg_rating difference
n_perms = 5000
perm_diffs = np.zeros(n_perms)
ratings = rated["avg_rating"].values
flags_quick = rated["quick"].values

for i in range(n_perms):
    shuffled = np.random.permutation(flags_quick)
    m_q = ratings[shuffled].mean()
    m_l = ratings[~shuffled].mean()
    perm_diffs[i] = m_q - m_l

p_val = np.mean(np.abs(perm_diffs) >= np.abs(obs_diff))

print(f"Mean avg_rating (quick) = {mean_quick:.3f}")
print(f"Mean avg_rating (long)  = {mean_long:.3f}")
print(f"Obs diff (quick – long)  = {obs_diff:.3f}")
print(f"P-value (two-sided)     = {p_val:.4f}")
# 4.3 Visualize null distribution of permuted differences
fig_null = px.histogram(
    x=perm_diffs,
    nbins=50,
    title="Null Distribution: Diff in avg_rating (quick – long)",
    labels={"x": "Permuted difference", "y": "Count"},
)
fig_null.add_vline(
    x=obs_diff,
    line_color="red",
    line_dash="dash",
    annotation_text="Observed difference",
    annotation_position="top right"
)
fig_null.update_layout(
    title_font=dict(size=24),
    xaxis=dict(title_font=dict(size=18), tickfont=dict(size=14)),
    yaxis=dict(title_font=dict(size=18), tickfont=dict(size=14))
)
fig_null.write_html("assets/plots/step4_null_avg_rating.html", include_plotlyjs="cdn")
print("Saved assets/plots/step4_null_avg_rating.html")

# 4.4 Boxplot of avg_rating by quick/long group
fig_box4 = px.box(
    rated,
    x=rated["quick"].map({True: "≤30 min", False: ">30 min"}),
    y="avg_rating",
    title="Avg Rating: Quick (≤30) vs Long (>30)",
    labels={"x": "Prep Time Group", "y": "Average Rating (stars)"},
    template="plotly_white",
    color_discrete_sequence=["#636EFA"]
)
fig_box4.update_layout(
    title_font=dict(size=24),
    xaxis=dict(title_font=dict(size=18), tickfont=dict(size=14)),
    yaxis=dict(title_font=dict(size=18), tickfont=dict(size=14)),
    showlegend=False
)
fig_box4.write_html("assets/plots/step4_box_avg_rating_by_quick.html", include_plotlyjs="cdn")
print("Saved assets/plots/step4_box_avg_rating_by_quick.html")

# 1) How big is the observed difference?
mean_quick = rated.loc[rated["quick"], "avg_rating"].mean()
mean_long  = rated.loc[~rated["quick"], "avg_rating"].mean()
obs_diff   = mean_quick - mean_long
print(f"Observed difference: {obs_diff:.3f}  (quick={mean_quick:.3f}, long={mean_long:.3f})")

# 2) What are the permuted‐difference statistics?
print("Permuted Δ mean rating summary:")
print(f"  mean   ≈ {perm_diffs.mean():.4f}")
print(f"  std    ≈ {perm_diffs.std(ddof=1):.4f}")
print(f"  95% CI ≈ [{np.percentile(perm_diffs, 2.5):.3f}, {np.percentile(perm_diffs, 97.5):.3f}]")

Mean avg_rating (quick) = 4.645
Mean avg_rating (long)  = 4.609
Obs diff (quick – long)  = 0.035
P-value (two-sided)     = 0.0000
Saved assets/plots/step4_null_avg_rating.html
Saved assets/plots/step4_box_avg_rating_by_quick.html
Observed difference: 0.035  (quick=4.645, long=4.609)
Permuted Δ mean rating summary:
  mean   ≈ -0.0001
  std    ≈ 0.0046
  95% CI ≈ [-0.009, 0.009]


## Step 5: Framing a Prediction Problem

In [6]:
# We want to predict each recipe’s avg_rating (continuous 1–5) – a regression problem.
# Response variable: avg_rating
# Evaluation metric: RMSE, because we measure average magnitude of errors.

# Use only features known at the time of posting:
# - minutes, calories, protein, n_ingredients

df = recipes.dropna(subset=["avg_rating"]).copy()
df = df[df["minutes"] <= 1440]

features = ["minutes", "calories", "protein", "n_ingredients"]
X = df[features]
y = df["avg_rating"]

# Split into train/test sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

print(f"Training set size: {X_train.shape[0]} rows")
print(f"Test set size:     {X_test.shape[0]} rows")

Training set size: 64504 rows
Test set size:     16127 rows


## Step 6: Baseline Model

In [7]:
# Baseline model using only 'minutes' and 'calories' with a RandomForestRegressor.

baseline_features = ["minutes", "calories"]
Xb_train = X_train[baseline_features]
Xb_test  = X_test[baseline_features]

baseline_model = RandomForestRegressor(n_estimators=100, random_state=42)
baseline_model.fit(Xb_train, y_train)

y_pred_baseline = baseline_model.predict(Xb_test)
baseline_rmse = mean_squared_error(y_test, y_pred_baseline, squared=False)

print("Baseline Model Results")
print("Features used: ['minutes', 'calories']")
print(f"Baseline RMSE = {baseline_rmse:.4f}")

Baseline Model Results
Features used: ['minutes', 'calories']
Baseline RMSE = 0.7362


/Users/naqotrunnadda/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.



## Step 7: Final Model

In [8]:
# We will add two more quantitative features: 'protein' and 'n_ingredients'.
# So final_features = ['minutes', 'calories', 'protein', 'n_ingredients'].
# We'll tune a RandomForestRegressor over a small grid of hyperparameters via GridSearchCV.

from sklearn.model_selection import GridSearchCV

final_features = ["minutes", "calories", "protein", "n_ingredients"]
Xf_train = X_train[final_features]
Xf_test  = X_test[final_features]

pipeline = Pipeline([
    ("rf", RandomForestRegressor(random_state=42))
])

param_grid = {
    "rf__n_estimators": [100],
    "rf__max_depth": [5],
    "rf__min_samples_split": [2]
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)
grid_search.fit(Xf_train, y_train)

best_params  = grid_search.best_params_
best_cv_rmse = -grid_search.best_score_

print("FINAL MODEL (GridSearchCV)")
print("Features used:", final_features)
print("Best hyperparameters:", best_params)
print(f"Best CV RMSE = {best_cv_rmse:.4f}")

final_model = grid_search.best_estimator_
y_pred_final = final_model.predict(Xf_test)
final_rmse = mean_squared_error(y_test, y_pred_final, squared=False)

print(f"Final Model Test RMSE = {final_rmse:.4f}\n")

fig_pred = px.scatter(
    x=y_test,
    y=y_pred_final,
    title="Predicted vs. Actual Avg Rating (Final Model)",
    labels={"x": "True avg_rating", "y": "Predicted avg_rating"}
)
fig_pred.update_layout(
    margin=dict(l=60, r=40, t=60, b=120),
    title_font=dict(size=24),
    xaxis=dict(title_font=dict(size=18), tickfont=dict(size=14)),
    yaxis=dict(title_font=dict(size=18), tickfont=dict(size=14)),
)
fig_pred.add_annotation(
    dict(
        xref="paper", yref="paper",
        x=0, y=-0.25,
        showarrow=False,
        align="left",
        font=dict(family="Monospace", size=14, color="#1B1F23"),
        text=(
            "This scatterplot compares final model predictions to true ratings. "
            "Points close to the diagonal indicate accurate predictions; "
            "spread around the line reflects prediction error (RMSE ≈ "
            f"{final_rmse:.3f})."
        )
    )
)
fig_pred.write_html("assets/plots/step7_predicted_vs_actual.html", include_plotlyjs="cdn")
print("Saved assets/plots/step7_predicted_vs_actual.html\n")

FINAL MODEL (GridSearchCV)
Features used: ['minutes', 'calories', 'protein', 'n_ingredients']
Best hyperparameters: {'rf__max_depth': 5, 'rf__min_samples_split': 2, 'rf__n_estimators': 100}
Best CV RMSE = 0.6390
Final Model Test RMSE = 0.6419

Saved assets/plots/step7_predicted_vs_actual.html



/Users/naqotrunnadda/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.



## Step 8: Fairness Analysis

In [9]:
# We ask: does the final model perform differently on “quick” vs. “long” recipes?
# Group X = long recipes (minutes > 30), Group Y = quick recipes (minutes ≤ 30).
# Evaluation metric: RMSE per group.
# Null Hypothesis (H0): RMSE_quick ≈ RMSE_long (any difference is random)
# Alternative Hypothesis (H1): RMSE_long > RMSE_quick (model worse on long recipes)

# Assemble test DataFrame
test_df = Xf_test.copy()
test_df["true"] = y_test.values
test_df["pred"] = y_pred_final
test_df["quick"] = test_df["minutes"] <= 30  # True=quick, False=long

# Compute observed RMSE per group
rmse_quick = mean_squared_error(
    test_df.loc[test_df["quick"], "true"],
    test_df.loc[test_df["quick"], "pred"],
    squared=False
)
rmse_long = mean_squared_error(
    test_df.loc[~test_df["quick"], "true"],
    test_df.loc[~test_df["quick"], "pred"],
    squared=False
)
obs_diff_rmse = rmse_long - rmse_quick

print("FAIRNESS ANALYSIS")
print(f"RMSE (quick recipes) = {rmse_quick:.4f}")
print(f"RMSE (long  recipes) = {rmse_long:.4f}")
print(f"Observed RMSE difference (long – quick) = {obs_diff_rmse:.4f}")

# Permutation test on group labels (n_permutations=1000)
n_perms = 1000
diffs = np.zeros(n_perms)
groups = test_df["quick"].values
y_true = test_df["true"].values
y_pred = test_df["pred"].values

for i in range(n_perms):
    permuted = rng.permutation(groups)
    rmse_q = mean_squared_error(
        y_true[permuted], y_pred[permuted], squared=False
    )
    rmse_l = mean_squared_error(
        y_true[~permuted], y_pred[~permuted], squared=False
    )
    diffs[i] = rmse_l - rmse_q

p_value_fair = np.mean(np.abs(diffs) >= np.abs(obs_diff_rmse))
print(f"P-value for RMSE difference = {p_value_fair:.4f}\n")

fig_fair = px.histogram(
    x=diffs,
    nbins=50,
    title="Null Dist: RMSE Difference (long – quick)",
    labels={"x": "Permuted RMSE Difference", "y": "Count"}
)
fig_fair.update_layout(
    margin=dict(l=60, r=40, t=60, b=120),
    title_font=dict(size=24),
    xaxis=dict(title_font=dict(size=18), tickfont=dict(size=14)),
    yaxis=dict(title_font=dict(size=18), tickfont=dict(size=14))
)
fig_fair.add_vline(
    x=obs_diff_rmse,
    line_color="red",
    line_dash="dash",
    annotation_text="Observed diff",
    annotation_position="top right"
)
fig_fair.add_annotation(
    dict(
        xref="paper", yref="paper",
        x=0, y=-0.25,
        showarrow=False,
        align="left",
        font=dict(family="Monospace", size=14, color="#1B1F23"),
        text=(
            "This histogram shows the permuted distribution of RMSE differences "
            "between long and quick recipes. The observed difference ("
            f"{obs_diff_rmse:.3f}) yields p = {p_value_fair:.4f}, suggesting "
            "no strong evidence of unfairness at α=0.05."
        )
    )
)
fig_fair.write_html("assets/plots/step8_null_rmse_difference.html", include_plotlyjs="cdn")
print("Saved assets/plots/step8_null_rmse_difference.html")

FAIRNESS ANALYSIS
RMSE (quick recipes) = 0.6261
RMSE (long  recipes) = 0.6550
Observed RMSE difference (long – quick) = 0.0289


/Users/naqotrunnadda/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/Users/naqotrunnadda/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/Users/naqotrunnadda/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/Users/naqotrunnadda/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and wi

P-value for RMSE difference = 0.1020

Saved assets/plots/step8_null_rmse_difference.html


/Users/naqotrunnadda/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/Users/naqotrunnadda/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/Users/naqotrunnadda/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/Users/naqotrunnadda/miniforge3/envs/dsc80/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and wi